# Notebook to test download of dicom images
*author: Wiebke Toussaint*

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import boto3
import tempfile
from time import time

projectdir = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(projectdir,'src'))

from d01_data.ingestion_xtdb import ingest_xtdb
from d02_intermediate.clean_xtdb import *
#from d00_utils.db_utils import *
#from d00_utils.s3_utils import *
from d02_intermediate.dcm_utils import decompress_dcm
from d00_utils.log_utils import get_logs

db_table = 'metadata'
credentials_file = os.path.join(os.path.expanduser('~'), '.psql_credentials.json')

In [ ]:
io_views = dbReadWriteViews()
io_views.list_tables()

### Download Sample Dicom Images

In [ ]:
instances = io_views.get_table('instances_w_labels_test_downsampleby5')
prefix = instances['studyidk'].astype(str) + '/a_' + instances['filename'].astype(str)
filenames = 'a_' + instances['studyidk'].astype(str) + '_' + instances['filename'].astype(str) + '.dcm'

download_dict = dict(zip(prefix, filenames))

In [ ]:
datadir = os.path.expanduser('/home/ubuntu/data/01_raw/test_downsampleby5')
raw_datadir = os.path.join(datadir, 'raw')

for p in list(download_dict.keys()):
    dcm_filepath=os.path.join(datadir, download_dict[p])
    dcm_rawfilepath=os.path.join(raw_datadir, download_dict[p]+'_raw')
   
    download_s3_objects('cibercv', outfile=dcm_filepath, prefix=p, suffix='.dcm') 
    decompress_dcm(dcm_filepath, dcm_rawfilepath)

In [ ]:
get_logs('d01_data', 'ERROR')

In [ ]:
def download_dcm_images():